In [24]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use("ggplot")
%matplotlib inline

In [25]:
class body:
    """defines a new class for bodies in our system
    Input:
    ------
    m
        mass of the body measured in kg
    r
        linear distance from the Sun (center of system) measured in km 
    th
        angular position measured in degrees
    v
        tangential velocity measured in km/s
    """
    
    def __init__(self,m,r,th,v):
        self.m = m
        self.r = r
        self.th = th
        self.v = v
        
        
Sun = body(1.989e30,0,0,0)
Venus = body(4.87e24,108.2e6,14.22,2.246991e2)
Earth = body(5.97e24,149.6e6,219.09,3.65200e2)
Mars = body(0.642e24,227.9e6,229.93,6.87e2)
Jupiter = body(1898e24,778.6e6,172.26,8.1660e2)
Saturn = body(568e24,1433.5e6,252.77,1.5145e3)
Titan = body(0.1345e24,1434.7e6,252.77,5.57)

#creates a list of all bodies
bodies = [Sun,Venus,Earth,Mars,Jupiter,Saturn,Titan]

#creates arrays for attributes of each body
m = np.zeros(len(bodies))
r = np.zeros_like(m)
v = np.zeros_like(m)
th = np.zeros_like(m)

for i in range(len(bodies)):
    m[i] = bodies[i].m
    r[i] = bodies[i].r
    v[i] = bodies[i].v
    th[i] = bodies[i].th

In [26]:
def position_components(th, r):
    th_rad = np.deg2rad(th)
    x = r*np.cos(th_rad)
    y = r*np.sin(th_rad)
    return np.array([x, y])

r_com = position_components(th,r)

In [41]:
dist_com = np.zeros((2,len(bodies),len(bodies)))
for i in range(2):
    for j in range(len(bodies)):
        for k in range(len(bodies)):
            if j != k:
                dist_com[i, j, k] = r_com[i, k]-r_com[i, j]

In [43]:
r,dist_com

(array([  3.41505099e+07,  -5.43071059e+10,   4.52041238e+13,
         -1.70139872e+10,  -1.71865935e+11,  -1.62665129e+10,
         -1.53379853e+11]),
 array([[[  0.00000000e+00,   1.04884715e+08,  -1.16113008e+08,
           -1.46704478e+08,  -7.71506501e+08,  -4.24614443e+08,
           -4.24969893e+08],
         [ -1.04884715e+08,   0.00000000e+00,  -2.20997723e+08,
           -2.51589194e+08,  -8.76391216e+08,  -5.29499159e+08,
           -5.29854609e+08],
         [  1.16113008e+08,   2.20997723e+08,   0.00000000e+00,
           -3.05914707e+07,  -6.55393493e+08,  -3.08501436e+08,
           -3.08856885e+08],
         [  1.46704478e+08,   2.51589194e+08,   3.05914707e+07,
            0.00000000e+00,  -6.24802023e+08,  -2.77909965e+08,
           -2.78265415e+08],
         [  7.71506501e+08,   8.76391216e+08,   6.55393493e+08,
            6.24802023e+08,   0.00000000e+00,   3.46892058e+08,
            3.46536608e+08],
         [  4.24614443e+08,   5.29499159e+08,   3.08501436e+08,

In [48]:
Fg = np.zeros_like(dist_com)
def F_grav(m1,m2,d):
    G_cnst = 6.6741e-20
    Fg = ((-G_cnst*m1*m2)*d/(np.linalg.norm(d))**3)
    return Fg

In [49]:
F_grav(m[0],m[1],dist_com[0, 1, 0])

5.8766800288674103e+19

In [50]:
for i in range(2):
    for j in range(len(bodies)):
        for k in range(len(bodies)):
            if j != k:
                Fg[i, j, k] = F_grav(m[j],m[k],dist_com[i, j, k])
Fg

array([[[  0.00000000e+00,  -5.87668003e+19,   5.87814201e+19,
           3.95982303e+18,   4.23296347e+20,   4.18202190e+20,
           9.88629236e+16],
        [  5.87668003e+19,   0.00000000e+00,   3.97301681e+13,
           3.29664917e+12,   8.03196572e+14,   6.58475252e+14,
           1.55715026e+11],
        [ -5.87814201e+19,  -3.97301681e+13,   0.00000000e+00,
           2.73338846e+14,   1.76059311e+15,   2.37794058e+15,
           5.61790964e+11],
        [ -3.95982303e+18,  -3.29664917e+12,  -2.73338846e+14,
           0.00000000e+00,   2.08323898e+14,   3.15114099e+14,
           7.44271790e+10],
        [ -4.23296347e+20,  -8.03196572e+14,  -1.76059311e+15,
          -2.08323898e+14,   0.00000000e+00,  -5.97927517e+17,
          -1.41877315e+14],
        [ -4.18202190e+20,  -6.58475252e+14,  -2.37794058e+15,
          -3.15114099e+14,   5.97927517e+17,   0.00000000e+00,
           4.03558693e+19],
        [ -9.88629236e+16,  -1.55715026e+11,  -5.61790964e+11,
          -7.

In [32]:
#calculates force of gravity between two bodies and creates an array of these forces
#example: Sun is bodies[0] and Earth is bodies[2], so gravity[2,0] is the force between Earth and Sun
grav = np.zeros((len(bodies),(len(bodies))))
grav_net = np.zeros_like(m)
pos = np.zeros_like(m)
vel = np.zeros_like(m)
acc = np.zeros_like(grav)
acc_net = np.zeros_like(m)

def F_grav(m1,m2,d):
    G_cnst = 6.6741e-5
    Fg = ((-G_cnst*m1*m2)/d**2)
    return Fg

#calculates force of gravity and acceleration due to each body
def gravity():
    for i in range(len(bodies)):
        for j in range(len(bodies)):
            if i != j and r[i] > r[j]:
                grav[i,j] = F_grav(m[i],m[j],(r[j]-r[i]))
                acc[i,j] = grav[i,j]/m[i]
            if i != j and r[i] < r[j]:
                grav[i,j] = -F_grav(m[i],m[j],(r[j]-r[i]))
                acc[i,j] = grav[i,j]/m[i]
            else:
                pass
        
        grav_net[i] = sum(grav[i])
        acc_net[i] = sum(acc[i])
        
    return grav_net, acc_net
grav_net, acc_net = gravity()

gx = np.zeros(len(bodies))
gy = np.zeros_like(gx)

def grav_components(g,th):
    th_rad = np.deg2rad(th)
    gx = g*np.cos(th_rad)
    gy = g*np.sin(th_rad)
    g_com = np.array([gx,gy])
    return g_com

In [33]:
g_com = np.zeros((len(bodies),2))
for i in range(len(bodies)):
    for j in range(len(bodies)):
        if i!=j:
            g_com[i] = F_grav(m[i],m[j],r[j]-r[i])

In [34]:
F_grav(m[0],m[1],r[0]-r[1])

-5.5220703140108166e+34

In [35]:
position_components(th[4],r[4]),position_components(th[0],r[0]),F_grav(m[4],m[0],r[4]-r[0]), \
(position_components(th[4],r[4])-position_components(th[0],r[0]))[1]


(array([ -7.71506501e+08,   1.04860283e+08]),
 array([ 0.,  0.]),
 -4.1561853001187886e+35,
 104860282.62248312)

In [36]:
position_components(th[1],r[1]),position_components(th[4],r[4])

(array([  1.04884715e+08,   2.65788724e+07]),
 array([ -7.71506501e+08,   1.04860283e+08]))

In [37]:
def velocity_components(th, v):
    """converts tangential velocity to cartesian components with respect to sun
    Input:
    ------
    th
        angle of planet in degrees
    v
        orbital velocity of selected planet
        
    Output:
    -------
    x, y
        x and y coordinates where sun is at origin
    """
    phi = th + 90
    vx = v*np.sin(phi)
    vy = v*np.cos(phi)
    v_com = np.array([vx, vy])
    return v_com

In [38]:
dt = 0.1
t_max = 10
Nsteps = int(t_max / dt)
time = dt*np.linspace(0, t_max, Nsteps)

def integrate_all_orbits(t_max=3650):
    """worry about description later""" 
    v_half = np.zeros(len(bodies))
    r_new = np.zeros(len(bodies))
    F_new = np.zeros(len(bodies))
    v_new = np.zeros(len(bodies))
    
    for t in time:
        for i in range(len(bodies)):
            v_half[i] = v[i] + 0.5*dt*grav_net[i]/m[i]
            r[i] = r[i] + dt*v_half[i]
            gravity()
            F_new[i] = grav_net[i]
            v[i] = v_half[i] + 0.5*dt*F_new[i]/m[i]
    
    return r, v

In [39]:
integrate_all_orbits(t_max = 10)

(array([  3.41505099e+07,  -5.43071059e+10,   4.52041238e+13,
         -1.70139872e+10,  -1.71865935e+11,  -1.62665129e+10,
         -1.53379853e+11]),
 array([  9.33431729e+06,  -5.48743177e+09,   4.61266590e+12,
         -1.74175592e+09,  -2.32238381e+10,  -3.46312418e+09,
         -1.78361352e+10]))